In [1]:
'''

{"en":
{"feeds":[
{"name":"system_information","url":"https://gbfs.baywheels.com/gbfs/en/system_information.json"},
{"name":"station_information","url":"https://gbfs.baywheels.com/gbfs/en/station_information.json"},
{"name":"station_status","url":"https://gbfs.baywheels.com/gbfs/en/station_status.json"},
{"name":"free_bike_status","url":"https://gbfs.baywheels.com/gbfs/en/free_bike_status.json"},
{"name":"system_hours","url":"https://gbfs.baywheels.com/gbfs/en/system_hours.json"},
{"name":"system_calendar","url":"https://gbfs.baywheels.com/gbfs/en/system_calendar.json"},
{"name":"system_regions","url":"https://gbfs.baywheels.com/gbfs/en/system_regions.json"},
{"name":"system_alerts","url":"https://gbfs.baywheels.com/gbfs/en/system_alerts.json"}]}
'''

'\n\n{"en":\n{"feeds":[\n{"name":"system_information","url":"https://gbfs.baywheels.com/gbfs/en/system_information.json"},\n{"name":"station_information","url":"https://gbfs.baywheels.com/gbfs/en/station_information.json"},\n{"name":"station_status","url":"https://gbfs.baywheels.com/gbfs/en/station_status.json"},\n{"name":"free_bike_status","url":"https://gbfs.baywheels.com/gbfs/en/free_bike_status.json"},\n{"name":"system_hours","url":"https://gbfs.baywheels.com/gbfs/en/system_hours.json"},\n{"name":"system_calendar","url":"https://gbfs.baywheels.com/gbfs/en/system_calendar.json"},\n{"name":"system_regions","url":"https://gbfs.baywheels.com/gbfs/en/system_regions.json"},\n{"name":"system_alerts","url":"https://gbfs.baywheels.com/gbfs/en/system_alerts.json"}]}\n'

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import json
#from uszipcode import Zipcode
import pandas as pd
import numpy as np
import googlemaps


In [3]:
#open up a firefox broswer and navigate to the webpage.
driver = webdriver.Firefox()

#send the browser to the jsin location
# this veribile has to be set to page_source
page_source = driver.get('https://gbfs.baywheels.com/gbfs/en/station_information.json')

#create a BeautifulSoup object from source json
soup = BeautifulSoup(driver.page_source)

#grabs body from json
stations = json.loads(soup.find("body").text)

#further refines the output, eliminates meta data. outputs a dict:
print(stations['ttl'])
stations = stations['data']['stations']

#closes webdrive
driver.close()

300


In [174]:
#put into pandas dataframe
df = pd.DataFrame(stations)

#get rid of columns that are not of interest to the project
df = df.drop(axis=1, labels = ['rental_methods', 'external_id', 'has_kiosk','eightd_has_key_dispenser', 'electric_bike_surcharge_waiver', 'eightd_station_services'])

#remove unused stations (unused defined as stations with 0 capcaity)
#df = df.loc[df['capacity'] != 0]

#dataframe of SF stations only
sf = df[df["lon"] < -122.371411]

df['station_id'] = df.station_id.astype('int64')


In [313]:
sf.to_csv('data/lyft/sf_stations.csv')

In [5]:
# #use google geoliocate API to convert lat/long of stations to addresss with zip code

# from geopy.geocoders import GoogleV3

# geolocator = GoogleV3(api_key='userkey') #creates geolocator object
# test_station = geolocator.reverse("37.78637526861584, -122.40490436553954") #station id 3



In [6]:
#takes the results of the google quarry above and saves to pd.dataframe
#google_results_df = pd.DataFrame(google_results)

#saves the pd.dataframe as .csv
#df.to_csv('data/lyft/stations_google_results.csv')

In [178]:
unique_trips = pd.read_csv('data/lyft/unique_trips')
unique_trips = unique_trips[['station1','station2']]

#add station1 lat lon
#df1 = unique_trips.merge(df[['station_id','lon','lat']], how='left', left_on ='station1', right_on = 'station_id')


#add station2 lat lon
df2 = df1.merge(df[['station_id','lon','lat']], how='left', left_on ='station2', right_on = 'station_id')

In [180]:
df2

,station1,station2,station_id_x,lon_x,lat_x,station_id_y,lon_y,lat_y
0,33,440,33.0,-122.415408,37.781650,440,-122.449257,37.765910
1,109,365,109.0,-122.421904,37.763316,365,-122.431946,37.780450
2,81,468,81.0,-122.393170,37.775880,468,-122.422953,37.777870
3,134,350,134.0,-122.420628,37.752428,350,-122.405787,37.771431
4,104,115,104.0,-122.390833,37.767045,115,-122.398773,37.765026
...,...,...,...,...,...,...,...,...
25420,52,477,52.0,-122.441838,37.777416,477,-122.476104,37.780720
25421,91,113,91.0,-122.398438,37.771762,113,-122.410345,37.764555
25422,54,359,54.0,-122.433274,37.777547,359,-122.398963,37.730168
25423,100,481,100.0,-122.410662,37.767100,481,-122.447374,37.781131


In [28]:
#Get bicycle travel duration estimates from google distance matrix api

# API_key = 'userkey'#enter Google Maps API key
# gmaps = googlemaps.Client(key=API_key)

#google_trip_results = []
# for i in df2.iterrows():
    
#     #print(i[1][7])
    
#     origin = (i[1][4],i[1][3])
#     destination = (i[1][7],i[1][6])
#     print(origin)
#     print(destination)
#     print("----")
    
#     result = (gmaps.distance_matrix(origin, destination, mode = 'bicycling', units = 'imperial'))
#     google_trip_results.append({'distance':result['rows'][0]['elements'][0]["distance"]["value"],'duration':result['rows'][0]['elements'][0]["duration"]["value"]})
#     print(f'Start station: {i[1][0]}, end station {i[1][1]})')
#     print(result['rows'][0]['elements'][0]["distance"]["value"]) #distance
#     print(result['rows'][0]['elements'][0]["duration"]["value"]) #duration

Start station: 33.0, end station 440.0)
4630
1431
Start station: 109.0, end station 365.0)
2734
840
Start station: 81.0, end station 468.0)
3963
1104
Start station: 134.0, end station 350.0)
3784
855
Start station: 104.0, end station 115.0)
1133
232
Start station: 39.0, end station 471.0)
2382
750
Start station: 85.0, end station 373.0)
4266
1124
Start station: 87.0, end station 459.0)
2953
698
Start station: 20.0, end station 384.0)
2656
888
Start station: 66.0, end station 66.0)
0
0
Start station: 375.0, end station 460.0)
6603
1548
Start station: 75.0, end station 400.0)
4792
1396
Start station: 458.0, end station 488.0)
12062
2546
Start station: 8.0, end station 87.0)
4579
1196
Start station: 356.0, end station 399.0)
5290
1475
Start station: 363.0, end station 413.0)
10956
2837
Start station: 56.0, end station 323.0)
4572
1132
Start station: 95.0, end station 98.0)
1294
383
Start station: 118.0, end station 129.0)
2713
662
Start station: 104.0, end station 463.0)
3695
706
Start st

Start station: 147.0, end station 441.0)
6974
1776
Start station: 43.0, end station 376.0)
3986
971
Start station: 6.0, end station 140.0)
7641
2011
Start station: 52.0, end station 506.0)
2777989
509443
Start station: 324.0, end station 359.0)
9120
2026
Start station: 24.0, end station 91.0)
2825
767
Start station: 100.0, end station 462.0)
6309
1237
Start station: 72.0, end station 359.0)
9238
1992
Start station: 381.0, end station 483.0)
3556
922
Start station: 42.0, end station 323.0)
3428
904
Start station: 134.0, end station 371.0)
6851
1543
Start station: 413.0, end station 424.0)
11903
2946
Start station: 39.0, end station 50.0)
5281
1307
Start station: 369.0, end station 436.0)
7977
1740
Start station: 85.0, end station 336.0)
2880
857
Start station: 20.0, end station 493.0)
2420
716
Start station: 66.0, end station 127.0)
4358
1143
Start station: 29.0, end station 361.0)
9286
2071
Start station: 89.0, end station 134.0)
3163
814
Start station: 52.0, end station 62.0)
3652
889

Start station: 390.0, end station 486.0)
9101
2483
Start station: 55.0, end station 462.0)
5155
1246
Start station: 438.0, end station 498.0)
6831
1700
Start station: 11.0, end station 108.0)
4768
1374
Start station: 343.0, end station 435.0)
6856
1596
Start station: 147.0, end station 412.0)
1872
512
Start station: 285.0, end station 502.0)
2777640
509410
Start station: 52.0, end station 487.0)
5891
1534
Start station: 77.0, end station 116.0)
2655
599
Start station: 24.0, end station 64.0)
1963
580
Start station: 10.0, end station 63.0)
2903
888
Start station: 33.0, end station 502.0)
2776866
509269
Start station: 381.0, end station 478.0)
5631
1642
Start station: 42.0, end station 364.0)
3410
813
Start station: 81.0, end station 386.0)
4884
1182
Start station: 321.0, end station 435.0)
7482
1727
Start station: 30.0, end station 453.0)
677
127
Start station: 323.0, end station 349.0)
2363
653
Start station: 413.0, end station 451.0)
2769205
507425
Start station: 90.0, end station 112

Start station: 364.0, end station 432.0)
4998
1161
Start station: 371.0, end station 500.0)
2764826
504716
Start station: 64.0, end station 412.0)
7467
1751
Start station: 13.0, end station 493.0)
2782
808
Start station: 112.0, end station 424.0)
5598
1638
Start station: 59.0, end station 147.0)
4074
987
Start station: 61.0, end station 481.0)
4511
1508
Start station: 451.0, end station 477.0)
2781523
506373
Start station: 70.0, end station 369.0)
3998
1053
Start station: 144.0, end station 499.0)
2773010
508360
Start station: 384.0, end station 386.0)
4994
1328
Start station: 3.0, end station 440.0)
5763
1636
Start station: 116.0, end station 471.0)
6546
2012
Start station: 432.0, end station 446.0)
7364
1520
Start station: 58.0, end station 456.0)
4671
1458
Start station: 120.0, end station 129.0)
1574
462
Start station: 55.0, end station 485.0)
4400
1164
Start station: 115.0, end station 134.0)
3252
815
Start station: 101.0, end station 359.0)
6832
1406
Start station: 138.0, end sta

Start station: 11.0, end station 38.0)
2339
459
Start station: 110.0, end station 496.0)
2774826
508753
Start station: 36.0, end station 499.0)
2763397
504432
Start station: 419.0, end station 461.0)
5234
1228
Start station: 43.0, end station 457.0)
2776632
509207
Start station: 6.0, end station 223.0)
5515
1465
Start station: 336.0, end station 349.0)
2937
678
Start station: 91.0, end station 453.0)
1693
334
Start station: 100.0, end station 285.0)
3383
1034
Start station: 10.0, end station 101.0)
4465
1205
Start station: 33.0, end station 460.0)
3900
962
Start station: 21.0, end station 355.0)
5390
1314
Start station: 81.0, end station 440.0)
7353
2084
Start station: 321.0, end station 457.0)
2776195
509114
Start station: 30.0, end station 499.0)
2775621
508858
Start station: 90.0, end station 134.0)
3593
918
Start station: 76.0, end station 373.0)
3683
937
Start station: 369.0, end station 453.0)
2536
683
Start station: 11.0, end station 482.0)
4378
1019
Start station: 87.0, end sta

...

Start station: 112.0, end station 506.0)
2774648
508731
Start station: 22.0, end station 459.0)
5410
1366
Start station: 121.0, end station 380.0)
5340
1562
Start station: 144.0, end station 453.0)
4722
990
Start station: 102.0, end station 116.0)
694
154
Start station: 49.0, end station 120.0)
4765
1379
Start station: 12.0, end station 324.0)
1796
694
Start station: 358.0, end station 450.0)
2768938
507500
Start station: 125.0, end station 461.0)
3514
755
Start station: 58.0, end station 390.0)
5187
1131
Start station: 441.0, end station 502.0)
2765678
504890
Start station: 390.0, end station 457.0)
2771828
508113
Start station: 399.0, end station 437.0)
12176
2472
Start station: 55.0, end station 471.0)
2586
892
Start station: 438.0, end station 453.0)
6210
1210
Start station: 115.0, end station 120.0)
2642
781
Start station: 78.0, end station 108.0)
1880
561
Start station: 486.0, end station 497.0)
2778928
509727
Start station: 110.0, end station 445.0)
3256
859
Start station: 147.0

Start station: 36.0, end station 73.0)
4353
1210
Start station: 112.0, end station 479.0)
3938
1476
Start station: 22.0, end station 44.0)
2278
608
Start station: 121.0, end station 359.0)
6546
1374
Start station: 144.0, end station 474.0)
9364
2358
Start station: 130.0, end station 133.0)
4638
1064
Start station: 72.0, end station 140.0)
4339
1074
Start station: 58.0, end station 483.0)
2133
521
Start station: 390.0, end station 482.0)
3349
682
Start station: 55.0, end station 498.0)
8598
2221
Start station: 25.0, end station 139.0)
4926
1245
Start station: 101.0, end station 144.0)
2645
617
Start station: 138.0, end station 490.0)
6915
1958
Start station: 11.0, end station 104.0)
4603
964
Start station: 343.0, end station 439.0)
5492
1520
Start station: 96.0, end station 368.0)
2615
923
Start station: 285.0, end station 506.0)
2777640
509410
Start station: 77.0, end station 120.0)
2268
672
Start station: 24.0, end station 124.0)
4566
1161
Start station: 10.0, end station 51.0)
6349
1

Start station: 28.0, end station 349.0)
2555
672
Start station: 60.0, end station 110.0)
1753
438
Start station: 23.0, end station 90.0)
2926
773
Start station: 364.0, end station 436.0)
4778
964
Start station: 25.0, end station 370.0)
1874
596
Start station: 371.0, end station 496.0)
2764826
504716
Start station: 13.0, end station 497.0)
2762879
504443
Start station: 36.0, end station 86.0)
3494
1037
Start station: 112.0, end station 436.0)
5132
1140
Start station: 352.0, end station 387.0)
3837
1178
Start station: 61.0, end station 485.0)
6317
1775
Start station: 107.0, end station 363.0)
4453
1180
Start station: 70.0, end station 381.0)
3606
883
Start station: 384.0, end station 398.0)
640
268
Start station: 3.0, end station 436.0)
7665
1774
Start station: 116.0, end station 459.0)
2533
536
Start station: 49.0, end station 54.0)
4138
1175
Start station: 72.0, end station 145.0)
4860
1194
Start station: 58.0, end station 460.0)
3878
890
Start station: 120.0, end station 125.0)
1791
4

Start station: 86.0, end station 433.0)
6965
1943
Start station: 19.0, end station 504.0)
2763331
504490
Start station: 365.0, end station 390.0)
6791
1594
Start station: 74.0, end station 488.0)
9690
2205
Start station: 457.0, end station 480.0)
2777938
505859
Start station: 60.0, end station 75.0)
2005
691
Start station: 23.0, end station 49.0)
1723
440
Start station: 71.0, end station 453.0)
4839
1158
Start station: 145.0, end station 323.0)
7735
1860
Start station: 454.0, end station 471.0)
1124
332
Start station: 4.0, end station 386.0)
4711
1165
Start station: 64.0, end station 445.0)
1533
443
Start station: 13.0, end station 460.0)
4678
1156
Start station: 36.0, end station 51.0)
5697
1761
Start station: 59.0, end station 370.0)
2056
681
Start station: 22.0, end station 122.0)
4466
1221
Start station: 352.0, end station 504.0)
2768253
507302
Start station: 107.0, end station 398.0)
4703
1543
Start station: 384.0, end station 419.0)
692
219
Start station: 349.0, end station 453.0

Start station: 457.0, end station 459.0)
2771200
504129
Start station: 23.0, end station 44.0)
2721
721
Start station: 362.0, end station 456.0)
11969
3043
Start station: 71.0, end station 480.0)
2238
632
Start station: 145.0, end station 358.0)
5702
1247
Start station: 454.0, end station 488.0)
12686
2714
Start station: 4.0, end station 431.0)
9429
2473
Start station: 73.0, end station 132.0)
3762
1056
Start station: 59.0, end station 361.0)
6977
1454
Start station: 22.0, end station 95.0)
4697
1340
Start station: 352.0, end station 477.0)
12692
3368
Start station: 121.0, end station 144.0)
2120
458
Start station: 349.0, end station 480.0)
3887
1179
Start station: 116.0, end station 413.0)
9861
2235
Start station: 49.0, end station 492.0)
4928
1516
Start station: 432.0, end station 500.0)
2769419
507617
Start station: 125.0, end station 345.0)
1795
438
Start station: 141.0, end station 468.0)
3657
873
Start station: 21.0, end station 142.0)
5943
1549
Start station: 97.0, end station 5

In [234]:
#Get public transit travel duratoin estimates from google distance matrix api

API_key = 'userkey'#enter Google Maps API key
gmaps = googlemaps.Client(key=API_key)

transit_dump = []
for i in df2.iterrows():
    
    #print(i[1][7])
    
    origin = (i[1][4],i[1][3])
    destination = (i[1][7],i[1][6])
#     print(origin)
#     print(destination)
#     print("----")
    
    result = (gmaps.distance_matrix(origin, destination, mode = 'transit', units = 'imperial'))
    transit_dump.append(result)
#     distance = result["rows"][0]["elements"][0]["distance"]["value"]
#     duration = result["rows"][0]["elements"][0]["duration"]["value"]
#     google_trip_results_transit.append({"distance": distance,"duration":duration})
#     print(f'Start station: {i[1][0]}, end station {i[1][1]})')
#     print(result['rows'][0]['elements'][0]["distance"]["value"]) #distance
#     print(result['rows'][0]['elements'][0]["duration"]["value"]) #duration
#     print("++++")

In [242]:
transit_dump[0]

{'destination_addresses': ['Carl St at Cole St, San Francisco, CA 94117, USA'],
 'origin_addresses': ['Golden Gate Ave at Hyde St, San Francisco, CA 94102, USA'],
 'rows': [{'elements': [{'distance': {'text': '2.5 mi', 'value': 3969},
     'duration': {'text': '15 mins', 'value': 915},
     'fare': {'currency': 'USD', 'text': '$3.00', 'value': 3},
     'status': 'OK'}]}],
 'status': 'OK'}

In [270]:
transit_dump[6917]

{'destination_addresses': ['37.77523487,-122.3974371'],
 'origin_addresses': ['37.795392,-122.394203'],
 'rows': [{'elements': [{'status': 'ZERO_RESULTS'}]}],
 'status': 'OK'}

In [294]:
transit_results = []
bad = []

for i in transit_dump:
    
    temp = i["rows"][0]["elements"][0]
      
    distance = temp.get("distance")
    if distance:
        distance = distance.get("value")
    else:
        bad.append(i)
        continue
    
    duration = temp.get("duration")
    duration = duration.get("value")

    transit_results.append((distance, duration))

In [65]:
df3 = df2[['station1', 'station2']]

In [123]:
#extract google api call for bicycle dictonary values into list (distance, duration)

list1 = []
for i in google_trip_results:
    distance = i['distance']
    duration = i['duration']
    list1.append((distance, duration))


In [299]:
#extract google api call for transit dictonary values into list (distance, duration)

list2 = []
for i in transit_results:
    distance = i[0]
    duration = i[1]
    list2.append((distance, duration))


In [41]:
#df2.to_csv('data/lyft/df2.csv')

In [10]:
#routes is a set of all possilbe paths between two stations within the SF stations dataframe
routes = set()

#adds unique trips to routes. unique is defined as 
for i in sf_stations.station_id:
    for j in sf_stations.station_id:
        if (int(j),int(i)) not in routes:
            routes.add((int(i),int(j)))

unique_trips = pd.DataFrame(routes, columns=['station1', 'station2'])

In [318]:
#unique_trips.to_csv('data/lyft/unique_trips')

In [175]:
#df.to_csv('data/lyft/stations.csv')

,station1,station2,station_id_x,lon_x,lat_x,station_id_y,lon_y,lat_y,dist_dur,distance,duration
0,33,440,33.0,-122.415408,37.781650,440.0,-122.449257,37.765910,"{'distance': 4630, 'duration': 1431}",4384,1379
1,109,365,109.0,-122.421904,37.763316,365.0,-122.431946,37.780450,"{'distance': 2734, 'duration': 840}",4384,1379
2,81,468,81.0,-122.393170,37.775880,468.0,-122.422953,37.777870,"{'distance': 3963, 'duration': 1104}",4384,1379
3,134,350,134.0,-122.420628,37.752428,350.0,-122.405787,37.771431,"{'distance': 3784, 'duration': 855}",4384,1379
4,104,115,104.0,-122.390833,37.767045,115.0,-122.398773,37.765026,"{'distance': 1133, 'duration': 232}",4384,1379
5,39,471,39.0,-122.436861,37.778999,471.0,-122.454622,37.777609,"{'distance': 2382, 'duration': 750}",4384,1379
6,85,373,85.0,-122.429156,37.770083,373.0,-122.405216,37.751792,"{'distance': 4266, 'duration': 1124}",4384,1379
7,87,459,87.0,-122.415674,37.769757,459.0,-122.405393,37.753123,"{'distance': 2953, 'duration': 698}",4384,1379
8,20,384,20.0,-122.399051,37.791300,384.0,-122.421568,37.794160,"{'distance': 2656, 'duration': 888}",4384,1379
9,66,66,66.0,-122.392741,37.778742,66.0,-122.392741,37.778742,"{'distance': 0, 'duration': 0}",4384,1379


In [301]:
df4 = pd.DataFrame(list1, columns=['bicycle_distance', 'bicycle_duration'])

In [303]:
df5 = pd.DataFrame(list2, columns=['transit_distance', 'transit_duration'])

In [304]:
frames = [df3, df4, df5]

df6 = pd.concat(frames, axis =1)

In [305]:
#df6.to_csv('data/lyft/df6.csv')